<div style="font-size:18pt; padding-top:20px; text-align:center"><b>Рекуррентная нейронная сеть и </b> <span style="font-weight:bold; color:green">TensorFlow</span></div><hr>
<div style="text-align:right;">Папулин С.Ю. <span style="font-style: italic;font-weight: bold;">(papulin_hse@mail.ru)</span></div>

<p>Подключение стилей оформления</p>

In [ ]:
%%html
<link href="css/style.css" rel="stylesheet" type="text/css">

In [ ]:
import inspect
import time

In [ ]:
import numpy as np
import tensorflow as tf

In [ ]:
from lib import rnn_lang_modeling_reader as reader

<a name="1"></a>
<div style="display:table; width:100%; padding-top:10px; padding-bottom:10px; border-bottom:1px solid lightgrey">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-size:14pt; font-weight:bold">1. Загрузка исходных данных</div>
    	<div style="display:table-cell; width:20%; text-align:center; background-color:whitesmoke; border:1px solid lightgrey"><a href="#0">К содержанию</a></div>
    </div>
</div>

<p><b>Вариант 1.</b> Из командной строки</p>

In [ ]:
!wget -P data/rnn-lang-modeling/ http://www.fit.vutbr.cz/~imikolov/rnnlm/simple-examples.tgz

In [ ]:
!tar xvf data/rnn-lang-modeling/simple-examples.tgz -C data/rnn-lang-modeling/

<p><b>Вариант 2.</b> Средствами Python</p>

In [ ]:
import urllib.request
import shutil
import os
import tarfile

In [ ]:
url = "http://www.fit.vutbr.cz/~imikolov/rnnlm/simple-examples.tgz"

filename = "data/rnn-lang-modeling/rnn-simple.tgz"

os.makedirs(os.path.dirname(filename), exist_ok=True)

# Загрузка архива
with urllib.request.urlopen(url) as response:
    with open(filename, 'wb') as output:
        shutil.copyfileobj(response, output)

# Распаковка
with tarfile.open(filename) as tar:
    tar.extractall(path="data/rnn-lang-modeling/")

Директория с исходными данными и для записи логов и модели

In [ ]:
data_path = "data/rnn-lang-modeling/simple-examples/data"
save_path = "log/rnn-lang-modeling/log"

Загрузка данных и преобразование в вектор индексов слов

In [ ]:
train_data, valid_data, test_data, vocabulary = reader.ptb_raw_data(data_path)

In [ ]:
train_data[:10]

Количество слов (токенов) в обучающем подмножестве

In [ ]:
len(train_data)

Индекс первого слова в тестовом подмножестве

In [ ]:
test_data[0]

Словарь преобразования слов в индексы

In [ ]:
vocabulary

In [ ]:
len(vocabulary)

Обратное пребразование индекса первого слова тестового подмножества

In [ ]:
for key in vocabulary:
    if vocabulary[key] == test_data[0]:
        print(key)

<a name="2"></a>
<div style="display:table; width:100%; padding-top:10px; padding-bottom:10px; border-bottom:1px solid lightgrey">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-size:14pt; font-weight:bold">2. Этапы построения сети</div>
    	<div style="display:table-cell; width:20%; text-align:center; background-color:whitesmoke; border:1px solid lightgrey"><a href="#0">К содержанию</a></div>
    </div>
</div>

#### Параметры модели

Структура сети

In [ ]:
num_steps = 24 # количество развертки LSTM
hidden_size = 200 # количество LSTM единиц
vocab_size = 10000 # размер словаря

Параметры инициализации весов и параметры обучения

In [ ]:
learning_rate = 1.0 # коэффициент скорости обучения
max_grad_norm = 5 # предельно допустимая норма градиента
max_epoch = 4 # количество обученных эпох с изначальным коэффициентом скорости обучения 
max_max_epoch = 13 # количество эпох
lr_decay = 0.5 # затухание скорости обучения для каждой эпохи после «max_epoch»
batch_size = 20 # размер batch
batch_size_test = 1 # размер batch при тестировании
num_steps_test = 1 # количество развертки LSTM при тестировании

In [ ]:
epoch_size_train = ((len(train_data) // batch_size) - 1) // num_steps # размер эпохи
epoch_size_valid = ((len(valid_data) // batch_size) - 1) // num_steps
epoch_size_test = ((len(test_data) // batch_size_test) - 1) // num_steps_test

#### Формирование исходных данных

In [ ]:
x_train, y_train = reader.ptb_producer(train_data, batch_size, num_steps, name="TrainInput")
x_valid, y_valid = reader.ptb_producer(valid_data, batch_size, num_steps, name="ValidInput")
x_test, y_test = reader.ptb_producer(test_data, batch_size_test, num_steps_test, name="TestInput")

In [ ]:
x_train

In [ ]:
y_train

#### Построение сети

Преобразование слов в распределенное представление (word2vec)

In [ ]:
embedding = tf.get_variable("embedding", [vocab_size, hidden_size], dtype=tf.float32)

In [ ]:
embedding

In [ ]:
inputs_train = tf.nn.embedding_lookup(embedding, x_train)
inputs_valid = tf.nn.embedding_lookup(embedding, x_valid)
inputs_test = tf.nn.embedding_lookup(embedding, x_test)

In [ ]:
inputs_train

Создание двух слоев LSTM

In [ ]:
lstm_layer_1 = tf.nn.rnn_cell.LSTMCell(hidden_size, forget_bias=0.0, 
                                            state_is_tuple=True,
                                            reuse=tf.get_variable_scope().reuse)
lstm_layer_2 = tf.nn.rnn_cell.LSTMCell(hidden_size, forget_bias=0.0, 
                                            state_is_tuple=True, 
                                            reuse=tf.get_variable_scope().reuse)

In [ ]:
lstm_layer_2

<b>Сотовая диаграмма LSTM</b>

<img src="images/rnn-lang-modeling/lstm_cell.png" width="670px">

<b>x<sub>t</sub></b> - входной вектор (слово/последовательность)

<b>h<sub>t-1</sub></b> - результат предыдущей соты h<sub>t-1</sub>

<b>s<sub>t</sub></b> - внутренняя переменная состояния t

<b>s<sub>t-1</sub></b> - внутренняя переменная состояния t-1 (создание эффекта повторения -> снижение вероятности исчезновения градиента)

<b>h<sub>t</sub></b> - выходной вектор


Объединение слоев в одну структуру

In [ ]:
multiple_cell = tf.nn.rnn_cell.MultiRNNCell([lstm_layer_1, lstm_layer_2], state_is_tuple=True)

<b>Развернутая рекуррентная нейронная сеть</b>

<img src="images/rnn-lang-modeling/rnn.png" width="696px">

На каждом временном шаге развертки LSTM подбирается новое слово, вывод h<sub>t-1</sub> предыдущей F соты подается в новую соту для определения следующего слова.

Инициализация начальных значений состояний

In [ ]:
initial_state_train = multiple_cell.zero_state(batch_size, tf.float32)
initial_state_valid = multiple_cell.zero_state(batch_size, tf.float32)
initial_state_test = multiple_cell.zero_state(batch_size_test, tf.float32)

In [ ]:
initial_state_train

Определение начальных и выходных состояний

In [ ]:
inputs_train = tf.unstack(inputs_train, num=num_steps, axis=1)
inputs_valid = tf.unstack(inputs_valid, num=num_steps, axis=1)
inputs_test = tf.unstack(inputs_test, num=num_steps_test, axis=1)

In [ ]:
inputs_train

In [ ]:
outputs_train, state_train = tf.nn.static_rnn(multiple_cell, inputs_train, initial_state=initial_state_train)
outputs_valid, state_valid = tf.nn.static_rnn(multiple_cell, inputs_valid, initial_state=initial_state_valid)
outputs_test, state_test = tf.nn.static_rnn(multiple_cell, inputs_test, initial_state=initial_state_test)

In [ ]:
len(outputs_train)

In [ ]:
state_train[0][1]

<b>LSTM архитектура сети</b>

<img src="images/rnn-lang-modeling/lstm_architecture.png" width="541px">

Размерность входных текстовых данных упорядочена следующим образом: (размер batch, количество развертки LSTM, количество LSTM единиц). 

Так для каждого batch и каждого слова в развертке LSTM существует вектор слоя внедрения длиной 200 для представления входного слова. Входные данные подаются в два «сложенных» слоя слотов LSTM. Вывод из этих развернутых слотов остается неизменным (размер batch, количество развертки LSTM, количество LSTM единиц).

Затем выходные данные передаются в полностью связанный слой Dense, на котором применяется функция активации softmax, возвращая массив вероятностных оценок. Оценки сравниваются с данными обучения y для каждой соты, затем выполняется обратное распространение ошибки и градиента. 

Так на каждом временном шаге модель пытается предсказать следующее следующее слово в последовательности.


Вычисление вероятности появления данных из y_train

In [ ]:
output_train = tf.reshape(tf.stack(axis=1, values=outputs_train), [-1, hidden_size])
output_valid = tf.reshape(tf.stack(axis=1, values=outputs_valid), [-1, hidden_size])
output_test = tf.reshape(tf.stack(axis=1, values=outputs_test), [-1, hidden_size])

In [ ]:
output_train

In [ ]:
softmax_w = tf.get_variable("softmax_w", [hidden_size, vocab_size], dtype=tf.float32)
softmax_b = tf.get_variable("softmax_b", [vocab_size], dtype=tf.float32)

Преобразование logits в трехмерный тензор для последовательности потерь

In [ ]:
logits_train = tf.matmul(output_train, softmax_w) + softmax_b
logits_valid = tf.matmul(output_valid, softmax_w) + softmax_b
logits_test = tf.matmul(output_test, softmax_w) + softmax_b

In [ ]:
logits_sample = tf.multinomial(logits_test, 1)

Функции потерь усредненные по batch

In [ ]:
loss_train = tf.contrib.legacy_seq2seq.sequence_loss_by_example([logits_train],
                                                          [tf.reshape(y_train, [-1])], 
                                                          [tf.ones([batch_size * num_steps],
                                                                   dtype=tf.float32)])

loss_valid = tf.contrib.legacy_seq2seq.sequence_loss_by_example([logits_valid],
                                                          [tf.reshape(y_valid, [-1])], 
                                                          [tf.ones([batch_size * num_steps],
                                                                   dtype=tf.float32)])

loss_test = tf.contrib.legacy_seq2seq.sequence_loss_by_example([logits_test],
                                                          [tf.reshape(y_test, [-1])], 
                                                          [tf.ones([batch_size_test * num_steps_test],
                                                                   dtype=tf.float32)])

In [ ]:
loss_train = tf.reduce_sum(loss_train) / batch_size
loss_valid = tf.reduce_sum(loss_valid) / batch_size
loss_test = tf.reduce_sum(loss_test) / batch_size_test

Конечное состояние

In [ ]:
final_state_train = state_train
final_state_valid = state_valid
final_state_test = state_test

Коэффициент обучения

In [ ]:
lr = tf.Variable(0.0, trainable=False)

In [ ]:
tvars = tf.trainable_variables()
grads, _ = tf.clip_by_global_norm(tf.gradients(loss_train, tvars), max_grad_norm)

Назначение оптимизатора (выполняться будет на batch)

In [ ]:
optimizer = tf.train.GradientDescentOptimizer(lr)
train_op = optimizer.apply_gradients(zip(grads, tvars),
                                           global_step=tf.train.get_or_create_global_step())

In [ ]:
new_lr = tf.placeholder(tf.float32, shape=[], name="new_learning_rate")
lr_update = tf.assign(lr, new_lr)

<a name="3"></a>
<div style="display:table; width:100%; padding-top:10px; padding-bottom:10px; border-bottom:1px solid lightgrey">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-size:14pt; font-weight:bold">3. Запуск обучения</div>
    	<div style="display:table-cell; width:20%; text-align:center; background-color:whitesmoke; border:1px solid lightgrey"><a href="#0">К содержанию</a></div>
    </div>
</div>

In [ ]:
sv = tf.train.Supervisor(logdir=save_path)

Функция для обучения модели

In [ ]:
def run_epoch(session, initial_state, loss, final_state, epoch_size, 
              num_steps, batch_size, eval_op=None, verbose=False):
    
    start_time = time.time()
    losses = 0.0
    iters = 0
    state = session.run(initial_state)

    fetches = {
      "loss": loss,
      "final_state": final_state,
    }
    if eval_op is not None:
        fetches["eval_op"] = eval_op

    for step in range(epoch_size):
        
        feed_dict = {}
        
        #состояние слота (state cell) и скртытое состояние (hidden state)
        for i, (c, h) in enumerate(initial_state):
            feed_dict[c] = state[i].c
            feed_dict[h] = state[i].h 
        
        vals = session.run(fetches, feed_dict)
        loss = vals["loss"]
        state = vals["final_state"]

        losses += loss
        iters += num_steps

        if verbose and step % (epoch_size // 10) == 10:
            print("%.3f perplexity: %.3f speed: %.0f wps" %
                (step * 1.0 / epoch_size, np.exp(losses / iters),
                 iters * batch_size / (time.time() - start_time)))

    return np.exp(losses / iters)

Запуск обучения

In [ ]:
with sv.managed_session() as session:
    for i in range(max_max_epoch):
        
        # затухание скорости обучения для каждой эпохи после «max_epoch»
        lr_decay = lr_decay ** max(i + 1 - max_epoch, 0.0)
        
        lr_value = learning_rate * lr_decay
        session.run(lr_update, feed_dict={new_lr: lr_value})
        
        print("Epoch: %d Learning rate: %.3f" % (i + 1, session.run(lr)))
        
        train_perplexity = run_epoch(session, initial_state_train, loss_train, final_state_train, epoch_size_train,
                                     num_steps, batch_size, eval_op=train_op, verbose=True)
        print("Train Perplexity: %.3f" % train_perplexity)
        
        valid_perplexity = run_epoch(session, initial_state_valid, loss_valid, final_state_valid, epoch_size_valid,
                                     num_steps, batch_size)
        print("Valid Perplexity: %.3f" % valid_perplexity + "\n")

    test_perplexity = run_epoch(session, initial_state_test, loss_test, final_state_test, epoch_size_test,
                                     num_steps_test, batch_size_test)
    print("Test Perplexity: %.3f" % test_perplexity)

Метрика <b>Perplexity</b> (растерянность) отражает распределение вероятностей предсказания объекта p и считается по следующей формуле:

<img src="images/rnn-lang-modeling/perplexity.png" width="231px">

В обработке естественного языка растерянность позволяет оценить языковые модели, подсчитывая обратную вероятность появления каждого последующего слова в сгенерированном тексте на основе распределения вероятностей обучающей выборки и выражается в двойке в положительной степени (чем ближе этот показатель к двум, тем точнее работает модель).

<a name="4"></a>
<div style="display:table; width:100%; padding-top:10px; padding-bottom:10px; border-bottom:1px solid lightgrey">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-size:14pt; font-weight:bold">4. Генерация текста</div>
    	<div style="display:table-cell; width:20%; text-align:center; background-color:whitesmoke; border:1px solid lightgrey"><a href="#0">К содержанию</a></div>
    </div>
</div>

Преобразование индексов в слова

In [ ]:
id_to_word = {}
for c, i in vocabulary.items():
    id_to_word[i] = c

Входная последовательность

In [ ]:
seed_text = np.array(vocabulary['raising']).reshape(1, 1)

Размер текста

In [ ]:
text_length = 200

Функция для генерации текста

In [ ]:
def generate_text(session, initial_state, final_state, logits_sample, input_data, feed, text_length):
    state = session.run(initial_state)
    fetches = {
        "final_state": final_state,
        "logits": logits_sample
    }
    
    generated_text = [feed]
    
    for i in range(text_length):
        feed_dict = {}
        feed_dict[input_data] = feed
        
        for i, (c, h) in enumerate(initial_state):
            feed_dict[c] = state[i].c
            feed_dict[h] = state[i].h
        
        vals = session.run(fetches, feed_dict)
        

        state = vals["final_state"]
        feed = vals["logits"]
        
      
        generated_text.append(feed)

    return generated_text

Сгенерированный текст на основе входной последовательности

In [ ]:
with sv.managed_session() as session:
    generated_text = generate_text(session, initial_state_test, final_state_test, logits_sample,
                                   x_test, np.array(seed_text).reshape(1, 1), text_length)
    generated_text = ' '.join([id_to_word[text[0, 0]] for text in generated_text])
    
    print(generated_text)

<a name="5"></a>
<div style="display:table; width:100%; padding-top:10px; padding-bottom:10px; border-bottom:1px solid lightgrey">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-size:14pt; font-weight:bold">5. Источники</div>
    	<div style="display:table-cell; width:20%; text-align:center; background-color:whitesmoke; border:1px solid lightgrey"><a href="#0">К содержанию</a></div>
    </div>
</div>

https://github.com/tensorflow/models/tree/master/tutorials/rnn